# Initialisation and setup

In [1]:
using Printf
using Dates
using LinearAlgebra
# Force BLAS to use 1 thread, allowing Julia threads to manage parallelism
BLAS.set_num_threads(1)


# 1. Load your local PolyhedraTree module
include("../geobin_jl/geobin.jl")

using .Geobin

In [4]:
using CSV
using DataFrames

function csv_to_matrices(filename)
    df = CSV.read(filename, DataFrame)
    # 1. Match the expected type: Dict{String, Any}
    model_params = Dict{String, Any}()

    layers = unique(df.layer)

    for l in layers
        # Filter data for weights and biases
        w_data = df[(df.layer .== l) .& (df.type .== "weight"), :]
        b_data = df[(df.layer .== l) .& (df.type .== "bias"), :]

        # Regex to pull numbers from "(r,c)" or "(r)"
        parse_coord(s) = parse.(Int, [m.match for m in eachmatch(r"\d+", s)])
        
        # Reconstruct Weight Matrix
        w_coords = parse_coord.(w_data.coord)
        rows = maximum(c -> c[1], w_coords) + 1
        cols = maximum(c -> c[2], w_coords) + 1
        
        # 2. Use Float64 to match your find_hyperplanes conversion
        W = zeros(Float64, rows, cols)
        for (c, v) in zip(w_coords, w_data.value)
            W[c[1]+1, c[2]+1] = v 
        end

        # Reconstruct Bias Vector
        b_coords = parse_coord.(b_data.coord)
        B = zeros(Float64, length(b_coords))
        for (c, v) in zip(b_coords, b_data.value)
            B[c[1]+1] = v
        end

        # 3. Use dot notation (layer_1.weight) to satisfy your replace() logic
        model_params[l * ".weight"] = W
        model_params[l * ".bias"] = B
    end
    
    return model_params
end
params = csv_to_matrices("dummy_weights.csv")
W, b = find_hyperplanes(params)

# Check the first layer
println("First layer shape: ", size(weights[1]))

First layer shape: (5, 2)


In [6]:
x0 = [-2.5; -5.0;;]

2×1 Matrix{Float64}:
 -2.5
 -5.0

# Activation pattern